In [1]:
# declare a list tasks whose products you want to use as inputs
upstream = ['fetch_securities']


In [2]:
# Parameters
upstream = {
    "fetch_securities": {
        "nb": "/Users/aiujdm2/market_watch/output/notebooks/fetch_securities.ipynb",
        "data": "/Users/aiujdm2/market_watch/output/data/raw/securities.csv",
    }
}
product = {
    "nb": "/Users/aiujdm2/market_watch/output/notebooks/normalize_security_names.ipynb",
    "data": "/Users/aiujdm2/market_watch/output/data/interim/normalized_security_names.csv",
}


In [3]:
import ast
import pandas as pd
import numpy as np
import json
from pathlib import Path
from src.utils import preprocess_text

In [4]:
securities_file_path = upstream['fetch_securities']['data']
securities_df = pd.read_csv(securities_file_path, index_col=0)

In [5]:
securities_df.drop_duplicates(subset=['cik'], inplace=True)

In [6]:
securities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9213 entries, 0 to 9212
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cik                    9213 non-null   int64  
 1   ticker                 9213 non-null   object 
 2   exchanges              9213 non-null   object 
 3   ein                    9203 non-null   float64
 4   full_name              9213 non-null   object 
 5   former_names           9213 non-null   object 
 6   short_name             499 non-null    object 
 7   gics_sector            499 non-null    object 
 8   gics_sub_industry      499 non-null    object 
 9   sic                    8428 non-null   float64
 10  sic_description        8414 non-null   object 
 11  headquarters_location  499 non-null    object 
dtypes: float64(2), int64(1), object(9)
memory usage: 935.7+ KB


In [7]:
securities_df.former_names.replace('[]', np.nan, inplace=True)
securities_df[securities_df.former_names.isnull()]

,cik,ticker,exchanges,ein,full_name,former_names,short_name,gics_sector,gics_sub_industry,sic,sic_description,headquarters_location
1,1843100,SAMA,"['Nasdaq', 'Nasdaq', 'Nasdaq']",861206818.0,Schultze Special Purpose Acquisition Corp. II,NaN,NaN,NaN,NaN,6770.0,Blank Checks,NaN
2,1527753,PSNL,['Nasdaq'],275411038.0,"Personalis, Inc.",NaN,NaN,NaN,NaN,8071.0,Services-Medical Laboratories,NaN
4,1437731,TGSGY,"['OTC', 'OTC']",0.0,TGS NOPEC Geophysical CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1816723,GRFX,"['', 'OTC']",0.0,Graphex Group Ltd,NaN,NaN,NaN,NaN,3620.0,Electrical Industrial Apparatus,NaN
7,1044378,BIOC,['Nasdaq'],800943522.0,BIOCEPT INC,NaN,NaN,NaN,NaN,8071.0,Services-Medical Laboratories,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9205,914156,UFPT,['Nasdaq'],42314970.0,UFP TECHNOLOGIES INC,NaN,NaN,NaN,NaN,3841.0,Surgical & Medical Instruments & Apparatus,NaN
9206,1778982,HUIZ,['Nasdaq'],0.0,Huize Holding Ltd,NaN,NaN,NaN,NaN,6411.0,"Insurance Agents, Brokers & Service",NaN
9207,1297184,AMPH,['Nasdaq'],330702205.0,"Amphastar Pharmaceuticals, Inc.",NaN,NaN,NaN,NaN,2834.0,Pharmaceutical Preparations,NaN
9208,56873,KR,['NYSE'],310345740.0,KROGER CO,NaN,Kroger,Consumer Staples,Food Retail,5411.0,Retail-Grocery Stores,"Cincinnati, Ohio"


In [8]:
def nomalize_names(securities_df):
    def extract_name(string):
        if len(str(string).strip()) > 5:
            string = ast.literal_eval(string)
            string = json.dumps(string)
            lst = json.loads(string)
            name = lst[0]["name"]
            return name
        else:
            return np.nan
            
    securities_df['former_name'] = securities_df.former_names.apply(extract_name)
    securities_df = securities_df.replace(to_replace=["/[A-Za-z]+/?"], value=[''], regex=True)
    securities_df = securities_df.replace(to_replace=[r'\\DE\\'], value=[''], regex=True)
    
    not_null_cond = ~securities_df['former_name'].isnull()
    
    securities_df.loc[not_null_cond,'former_name'] = preprocess_text(securities_df.loc[not_null_cond,'former_name'])
    
    securities_df['full_name'] = preprocess_text(securities_df.full_name)
    securities_df.replace('[]', np.nan, inplace=True)

    columns_to_select = ['cik','ticker','full_name', 'former_name']
    
    return securities_df[columns_to_select]

In [9]:
securities_df = nomalize_names(securities_df)

In [10]:
output_file_path = product['data']
Path(output_file_path).parent.mkdir(exist_ok=True, parents=True)
securities_df.to_csv(output_file_path)
print(f"Saved file {output_file_path}")

Saved file /Users/aiujdm2/market_watch/output/data/interim/normalized_security_names.csv
